In [1]:
from pandas import DataFrame
import pandas as pd
import numpy as np
import git
import matplotlib.pyplot as plt
import random
import os
from scipy.stats import wilcoxon
from sqlalchemy import create_engine, or_, Column, Integer, String, Float, DateTime, ForeignKey, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from do_analysis import Experiment, ExperimentSet

In [2]:
def from_db_to_pandas(query):
    last = ""
    headers = []
    datasets = []
    series = []
    serie = None
    code_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
    repo = git.Repo(code_directory)

    for experiment in query:
        if last != experiment.file_name:
            last = experiment.file_name
            dataset_name = experiment.file_name.rsplit('/')[-1].split(".")[0]
            datasets.append(dataset_name)
            if serie is not None:
                series.append(serie)
            serie = []
        message = repo.commit(experiment.set.commit).message.strip()
        message = experiment.set.description
        measure =  experiment.method.split('.')[-1]
        if "LearningBased" in measure:
            strategy = measure.split(" ")[4]
            weight = measure.split(" ")[6]
            measure = f"LearningBased strategy {strategy}, weight {weight}"
        header = f"{measure} - {message} - {experiment.set.commit}"
        if header not in headers:
            headers.append(header)
        if experiment.number_of_classes is None or experiment.number_of_clusters != experiment.number_of_classes:
            serie.append(np.nan)
        else:
            serie.append(experiment.f_score)


    series.append(serie)

    return pd.DataFrame(series, index = datasets, columns=headers)


In [3]:
engine = create_engine('sqlite:///results.db')
session_class = sessionmaker(bind=engine)
session = session_class()
query = session.query(Experiment).order_by(Experiment.file_name, Experiment.id)
#query = for experiment in session.query(Experiment).filter(or_(Experiment.set_id==i for i in [17])).order_by(Experiment.file_name):
#query = for experiment in session.query(Experiment).filter(Experiment.number_of_clusters == Experiment.number_of_classes).order_by(Experiment.file_name):
df = from_db_to_pandas(query)
df

,"LearningBased strategy A, weight N - LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3","LearningBased strategy A, weight K - LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3","LearningBased strategy A, weight A - LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3","LearningBased strategy B, weight N - LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3","LearningBased strategy B, weight K - LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3","LearningBased strategy B, weight A - LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3","LearningBased strategy C, weight N - LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3","LearningBased strategy C, weight K - LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3","LearningBased strategy C, weight A - LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3","LearningBased strategy D, weight N - LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3",...,"LearningBased strategy E, weight A - LearningBased times (1 -kappa) sending 0 when kappa is low - 042c4faa6d24896c305a2636002a1660ed034b39","LearningBased strategy N, weight N - LearningBased times (1 -kappa) sending 0 when kappa is low - 042c4faa6d24896c305a2636002a1660ed034b39","LearningBased strategy N, weight K - LearningBased times (1 -kappa) sending 0 when kappa is low - 042c4faa6d24896c305a2636002a1660ed034b39","LearningBased strategy N, weight A - LearningBased times (1 -kappa) sending 0 when kappa is low - 042c4faa6d24896c305a2636002a1660ed034b39",EskinModified - Original measures times (1 - kappa) sending 0 when kappa is low - 042c4faa6d24896c305a2636002a1660ed034b39,GambaryanModified - Original measures times (1 - kappa) sending 0 when kappa is low - 042c4faa6d24896c305a2636002a1660ed034b39,GoodallModified - Original measures times (1 - kappa) sending 0 when kappa is low - 042c4faa6d24896c305a2636002a1660ed034b39,OFModified - Original measures times (1 - kappa) sending 0 when kappa is low - 042c4faa6d24896c305a2636002a1660ed034b39,IOFModified - Original measures times (1 - kappa) sending 0 when kappa is low - 042c4faa6d24896c305a2636002a1660ed034b39,LinModified - Original measures times (1 - kappa) sending 0 when kappa is low - 042c4faa6d24896c305a2636002a1660ed034b39
allbp_complete_cleaned,NaN,NaN,0.653814,0.792902,0.790311,0.601846,0.601846,0.693988,0.601846,0.829734,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
arrhythmia_cleaned,NaN,NaN,NaN,0.412789,0.412812,0.416614,0.428022,0.430032,0.441823,0.427121,...,0.509902,NaN,NaN,NaN,0.504150,NaN,NaN,0.505309,0.508722,NaN
australian_cleaned,0.585064,0.674492,0.580529,0.674492,0.674492,0.572705,0.674492,0.727323,0.727323,0.636628,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cervical_cancer_biopsy_cleaned,0.926908,0.926908,0.926908,0.926908,0.926908,0.926908,0.926908,0.926908,0.926908,0.926908,...,0.926908,0.926908,0.926908,0.926908,0.926908,NaN,NaN,0.926908,0.926908,NaN
cervical_cancer_cytology_cleaned,0.935389,0.935389,0.935389,0.935389,0.935389,0.935389,0.935389,0.935389,0.935389,0.935389,...,0.935389,0.935389,0.935389,0.935389,0.935389,NaN,NaN,0.935389,0.935389,NaN
cervical_cancer_hinselmann_cleaned,0.945911,0.945911,0.945911,0.945911,0.945911,0.945911,0.945911,0.945911,0.945911,0.945911,...,0.945911,0.945911,0.945911,0.945911,0.945911,NaN,NaN,0.945911,0.945911,NaN
cervical_cancer_schiller_cleaned,0.906946,0.906946,0.906946,0.906946,0.906946,0.906946,0.906946,0.906946,0.906946,0.906946,...,0.906946,0.906946,0.906946,0.906946,0.906946,NaN,NaN,0.906946,0.906946,NaN
crx_cleaned,0.674584,0.674492,0.674584,0.674584,0.674584,0.674492,0.674584,0.674492,0.674492,0.674492,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
diagnosis1_cleaned,0.567037,0.564785,0.567037,0.567037,0.564785,0.564785,0.567037,0.564785,0.564785,0.567037,...,0.615792,0.567037,0.567037,0.567037,0.615792,0.722096,0.722096,0.615792,0.615792,0.615792
diagnosis2_cleaned,0.643799,0.744626,0.643799,0.643799,0.74

In [4]:
dft = df.apply(lambda x: x.rank(ascending=False), axis = 1)
dft

,"LearningBased strategy A, weight N - LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3","LearningBased strategy A, weight K - LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3","LearningBased strategy A, weight A - LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3","LearningBased strategy B, weight N - LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3","LearningBased strategy B, weight K - LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3","LearningBased strategy B, weight A - LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3","LearningBased strategy C, weight N - LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3","LearningBased strategy C, weight K - LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3","LearningBased strategy C, weight A - LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3","LearningBased strategy D, weight N - LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3",...,"LearningBased strategy E, weight A - LearningBased times (1 -kappa) sending 0 when kappa is low - 042c4faa6d24896c305a2636002a1660ed034b39","LearningBased strategy N, weight N - LearningBased times (1 -kappa) sending 0 when kappa is low - 042c4faa6d24896c305a2636002a1660ed034b39","LearningBased strategy N, weight K - LearningBased times (1 -kappa) sending 0 when kappa is low - 042c4faa6d24896c305a2636002a1660ed034b39","LearningBased strategy N, weight A - LearningBased times (1 -kappa) sending 0 when kappa is low - 042c4faa6d24896c305a2636002a1660ed034b39",EskinModified - Original measures times (1 - kappa) sending 0 when kappa is low - 042c4faa6d24896c305a2636002a1660ed034b39,GambaryanModified - Original measures times (1 - kappa) sending 0 when kappa is low - 042c4faa6d24896c305a2636002a1660ed034b39,GoodallModified - Original measures times (1 - kappa) sending 0 when kappa is low - 042c4faa6d24896c305a2636002a1660ed034b39,OFModified - Original measures times (1 - kappa) sending 0 when kappa is low - 042c4faa6d24896c305a2636002a1660ed034b39,IOFModified - Original measures times (1 - kappa) sending 0 when kappa is low - 042c4faa6d24896c305a2636002a1660ed034b39,LinModified - Original measures times (1 - kappa) sending 0 when kappa is low - 042c4faa6d24896c305a2636002a1660ed034b39
allbp_complete_cleaned,NaN,NaN,101.5,69.0,82.5,133.0,133.0,86.5,133.0,45.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
arrhythmia_cleaned,NaN,NaN,NaN,151.0,148.5,120.5,81.0,72.5,22.5,90.0,...,2.0,NaN,NaN,NaN,6.0,NaN,NaN,5.0,4.0,NaN
australian_cleaned,183.0,73.0,188.5,73.0,73.0,201.5,73.0,24.0,24.0,149.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cervical_cancer_biopsy_cleaned,133.0,133.0,133.0,133.0,133.0,133.0,133.0,133.0,133.0,133.0,...,133.0,133.0,133.0,133.0,133.0,NaN,NaN,133.0,133.0,NaN
cervical_cancer_cytology_cleaned,134.0,134.0,134.0,134.0,134.0,134.0,134.0,134.0,134.0,134.0,...,134.0,134.0,134.0,134.0,134.0,NaN,NaN,134.0,134.0,NaN
cervical_cancer_hinselmann_cleaned,138.0,138.0,138.0,138.0,138.0,138.0,138.0,138.0,138.0,138.0,...,138.0,138.0,138.0,138.0,138.0,NaN,NaN,138.0,138.0,NaN
cervical_cancer_schiller_cleaned,142.0,142.0,142.0,142.0,142.0,142.0,142.0,142.0,142.0,142.0,...,142.0,142.0,142.0,142.0,142.0,NaN,NaN,142.0,142.0,NaN
crx_cleaned,55.0,144.5,55.0,55.0,55.0,144.5,55.0,144.5,144.5,144.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
diagnosis1_cleaned,186.0,268.5,186.0,186.0,268.5,268.5,186.0,268.5,268.5,186.0,...,92.5,186.0,186.0,186.0,92.5,14.5,14.5,92.5,92.5,92.5
diagnosis2_cleaned,143.0,55.5,143.0,143.0,55.5,55.5,143.0,55.5,55.5,143.0,...,253.0,143.0,143.0,143.0,253.0,336.5,336.5,253.0,253.0,253.0


In [5]:
averages = dft.mean()
averages.sort_values()

LinModified_KappaMax - Original measures - 0bf5188132e1744054ce5a9972258bdb005832a3                                                                           30.425926
LearningBased strategy C, weight K - LearningBased times 1 - kappa - 59eb8169a0aeeb91c3c70b1923b4b1275f7179b3                                                 45.241379
GambaryanModified - Original measures times auc sending max dissimilarity when auc is low - 3ac62e7b244a96c0362e823c17c32cf39a78da3c                          46.020833
GambaryanModified - Original measures times AUC - a0007a6a67f9ef6d2aa86b22415c27038d6b2c0e                                                                    46.020833
LearningBased strategy D, weight N - LearningBased times kappa - faef3e477809ebfa1bccb7d834593c449765f360                                                     46.327586
LearningBased strategy D, weight K - LearningBased times kappa - faef3e477809ebfa1bccb7d834593c449765f360                                                     46

In [6]:
strategies = {}
for i, average in enumerate(averages):
    commit = averages.index[i].split(" - ", 1)[-1]
    if commit in strategies:
        items, avg_position = strategies[commit]
        accumulated = avg_position * items
        accumulated = accumulated + average
        items =  items + 1
        strategies[commit] = (items, accumulated/items)
    else:
        strategies[commit] = (1, average)
strategies

{'LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3': (18,
  108.12983425064364),
 'Original measures - 0bf5188132e1744054ce5a9972258bdb005832a3': (13,
  144.44781184651876),
 'Original measures times kappa - faef3e477809ebfa1bccb7d834593c449765f360': (6,
  139.73845443349754),
 'LearningBased times kappa - faef3e477809ebfa1bccb7d834593c449765f360': (18,
  126.95442209450832),
 'LearningBased times AUC - a0007a6a67f9ef6d2aa86b22415c27038d6b2c0e': (18,
  112.01701902025178),
 'Original measures times AUC - a0007a6a67f9ef6d2aa86b22415c27038d6b2c0e': (6,
  159.2152777777778),
 'Original measures times 1 - AUC - 1e65eb018840cfaa74fb4308d82a7b9ad171036a': (6,
  185.38464696223318),
 'LearningBased times 1 - AUC - 1e65eb018840cfaa74fb4308d82a7b9ad171036a': (18,
  136.58668455675164),
 'LearningBased times 1 - kappa - 59eb8169a0aeeb91c3c70b1923b4b1275f7179b3': (18,
  108.70995880095498),
 'Original measures times 1 - kappa - 59eb8169a0aeeb91c3c70b1923b4b1275f7179b3': (6,
  197.15

In [8]:
def only_upper(s):
    return "".join(c for c in s if c.isupper())

strategies = {}
measure = {}
for i, average in enumerate(averages):
    measure, commit = averages.index[i].split(" - ", 1)
    if "OccurenceFrequency" in measure:
        measure = only_upper(measure)
    if commit in strategies:
        strategies[commit][measure] = average
    else:
        strategies[commit] = {measure: average}

In [9]:
for strategy in strategies:
    other_strategies = [x for x in strategies if x != strategy]
    better_results = 0
    for measure in strategies[strategy]:
        avg = strategies[strategy][measure]
        for other_strategy in other_strategies:
            if measure in strategies[other_strategy]:
                other_avg = strategies[other_strategy][measure]
                if avg < other_avg:
                    better_results = better_results +1
    num_of_measures = len(strategies[strategy])
    print(f"Strategy {strategy} has {better_results} better results, for {num_of_measures} measures, which averages to {better_results/num_of_measures} better results per measure")

Strategy LearningBased Basic - 0bf5188132e1744054ce5a9972258bdb005832a3 has 120 better results, for 18 measures, which averages to 6.666666666666667 better results per measure
Strategy Original measures - 0bf5188132e1744054ce5a9972258bdb005832a3 has 2 better results, for 13 measures, which averages to 0.15384615384615385 better results per measure
Strategy Original measures times kappa - faef3e477809ebfa1bccb7d834593c449765f360 has 60 better results, for 6 measures, which averages to 10.0 better results per measure
Strategy LearningBased times kappa - faef3e477809ebfa1bccb7d834593c449765f360 has 85 better results, for 18 measures, which averages to 4.722222222222222 better results per measure
Strategy LearningBased times AUC - a0007a6a67f9ef6d2aa86b22415c27038d6b2c0e has 117 better results, for 18 measures, which averages to 6.5 better results per measure
Strategy Original measures times AUC - a0007a6a67f9ef6d2aa86b22415c27038d6b2c0e has 47 better results, for 6 measures, which average